### Data Merging

The purpose of this notebook is to combine the data sets that will be used in this project into a single file that will be saved as a csv. The first data source is the mRFEI Index which is a continious measure of food accessibility. The unit of observation is the census tract; each census tract is assigned a score that represents that share of healthful food retailers (out of all food retailers) in a tract. The higher the score, the more healthful the tract is. This measure is the source of the outcome/target variables. Therefore, all subsequent data sets will be merged onto this data set. The second data source is the American Community Survey (ACS). 20 variables that were identified by Amin et. al. will be downloaded and merged onto the mRFEI Index. The resulting dataframe will consist of demographic and economic characteristics for all the census tracts in the mRFEI Index. 

In [6]:
## import required packages
import numpy as np
import pandas as pd 
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
import matplotlib.ticker as mtick
from matplotlib.ticker import FormatStrFormatter, StrMethodFormatter
import missingno as ms 

### Target

In [7]:
## load mREFI data file [i.e, contains the outcome/target variable]
mREFI=pd.read_excel("data/2_16_mrfei_data_table-2.xls", converters={'fips': str})#specify fips dtype to keep leading 0s
mREFI.shape

(65345, 3)

In [8]:
## view dataframe
mREFI.head()

,state,fips,mrfei
0,AL,01001020100,16.6667
1,AL,01001020200,16.6667
2,AL,01001020300,17.6471
3,AL,01001020400,13.3333
4,AL,01001020500,6.25


To merge the ACS variables onto the mREFI data, the census tract FIPS codes in both datasets need to have the same format. To make the mREFI FIPS code match with the ACS format, a leading string expression (i.e, 1400000US) will be added. 

In [9]:
## add leading string [1400000US] to match with acs format 
mREFI['fips_modified']=mREFI['fips'].apply(lambda x: '1400000US' + str(x))
mREFI.head()

,state,fips,mrfei,fips_modified
0,AL,01001020100,16.6667,1400000US01001020100
1,AL,01001020200,16.6667,1400000US01001020200
2,AL,01001020300,17.6471,1400000US01001020300
3,AL,01001020400,13.3333,1400000US01001020400
4,AL,01001020500,6.25,1400000US01001020500


### Features

**`Transportation`**: we will compute the share of the population using public transportation and merge this variable onto the mRFEI Index.

In [10]:
## import data set
transport=pd.read_csv('data/acs_5year_2010/transport.csv', skiprows=[1])
print(transport.shape)

(74002, 114)


In [11]:
## keep only variables needed to compute % of population using public transportion
transport=transport.filter(['GEO_ID', 'B08101_025E','B08101_001E', 'NAME'])
transport.head()

,GEO_ID,B08101_025E,B08101_001E,NAME
0,1400000US01001020100,42,903,"Census Tract 201, Autauga County, Alabama"
1,1400000US01001020200,0,809,"Census Tract 202, Autauga County, Alabama"
2,1400000US01001020300,14,1542,"Census Tract 203, Autauga County, Alabama"
3,1400000US01001020400,0,2192,"Census Tract 204, Autauga County, Alabama"
4,1400000US01001020500,0,5091,"Census Tract 205, Autauga County, Alabama"


#### Create attribute that represents % of population using public transportation (excluding taxicab)

**B08101_025E**: Number of workers 16 years and over using public transportation (excluding taxicab)<br>
**B08101_001E**: Number of workers 16 years and over

In [12]:
#--divide number of people using public transport by total population-------
transport['perc_public_transport']=(transport['B08101_025E']/transport['B08101_001E'])*100


In [13]:
#check to make sure there aren't any tracts with a value for zero in the denominator[i.e, workers 16 years and over]
transport.loc[transport.B08101_001E==0]

,GEO_ID,B08101_025E,B08101_001E,NAME,perc_public_transport
333,1400000US01003990000,0,0,"Census Tract 9900, Baldwin County, Alabama",NaN
398,1400000US01015981903,0,0,"Census Tract 9819.03, Calhoun County, Alabama",NaN
868,1400000US01097990000,0,0,"Census Tract 9900, Mobile County, Alabama",NaN
1063,1400000US01117980000,0,0,"Census Tract 9800, Shelby County, Alabama",NaN
1223,1400000US02185000300,0,0,"Census Tract 3, North Slope Borough, Alaska",NaN
...,...,...,...,...,...
73950,1400000US72113993000,0,0,"Census Tract 9930, Ponce Municipio, Puerto Rico",NaN
73955,1400000US72115990000,0,0,"Census Tract 9900, Quebradillas Municipio, Pue...",NaN
73959,1400000US72117990400,0,0,"Census Tract 9904, Rinc?n Municipio, Puerto Rico",NaN
73971,1400000US72119992700,0,0,"Census Tract 9927, R?o Grande Municipio, Puert...",NaN


Note that having a denominator of zero essentially means that the census tract does not have any residents who are 16 years or older. The goal of this study is to study tracts that are inhabited. Therefore, observations without any individuals who are 16 years or older will be dropped. 

In [14]:
## keep only observations/tracts that are inhabited (i.e, denominator is not equal to zero)
transport=transport.loc[transport.B08101_001E!=0]
transport.head()

,GEO_ID,B08101_025E,B08101_001E,NAME,perc_public_transport
0,1400000US01001020100,42,903,"Census Tract 201, Autauga County, Alabama",4.651163
1,1400000US01001020200,0,809,"Census Tract 202, Autauga County, Alabama",0.000000
2,1400000US01001020300,14,1542,"Census Tract 203, Autauga County, Alabama",0.907912
3,1400000US01001020400,0,2192,"Census Tract 204, Autauga County, Alabama",0.000000
4,1400000US01001020500,0,5091,"Census Tract 205, Autauga County, Alabama",0.000000


In [15]:
## remove columns used to generate 'perc_public_transport' as they are not needed anymore
transport=transport.drop(columns=['B08101_025E', 'B08101_001E'])
transport.head()

,GEO_ID,NAME,perc_public_transport
0,1400000US01001020100,"Census Tract 201, Autauga County, Alabama",4.651163
1,1400000US01001020200,"Census Tract 202, Autauga County, Alabama",0.000000
2,1400000US01001020300,"Census Tract 203, Autauga County, Alabama",0.907912
3,1400000US01001020400,"Census Tract 204, Autauga County, Alabama",0.000000
4,1400000US01001020500,"Census Tract 205, Autauga County, Alabama",0.000000


In [16]:
## merge transport variable onot the mREFI dataframe
mREFI=pd.merge(left=mREFI, right=transport, how='left', left_on='fips_modified', right_on='GEO_ID', indicator=True)


In [17]:
## check left_only merges
mREFI.loc[mREFI['_merge']=='left_only']

,state,fips,mrfei,fips_modified,GEO_ID,NAME,perc_public_transport,_merge
7,AL,01001020800,0,1400000US01001020800,NaN,NaN,NaN,left_only
22,AL,01003010901,41.1765,1400000US01003010901,NaN,NaN,NaN,left_only
23,AL,01003010902,18.1818,1400000US01003010902,NaN,NaN,NaN,left_only
25,AL,01003011100,16.6667,1400000US01003011100,NaN,NaN,NaN,left_only
31,AL,01003011404,15.7895,1400000US01003011404,NaN,NaN,NaN,left_only
...,...,...,...,...,...,...,...,...
65312,WY,56029995500,12.5,1400000US56029995500,NaN,NaN,NaN,left_only
65321,WY,56035000100,22.2222,1400000US56035000100,NaN,NaN,NaN,left_only
65334,WY,56039997600,25,1400000US56039997600,NaN,NaN,NaN,left_only
65335,WY,56039997700,10,1400000US56039997700,NaN,NaN,NaN,left_only


NOTE: there are 14591 census tracts that are not present in the transportation dataset from ACS. We will retain them for now (they will have Nas for perc_public_transport) and decide whether to impute or drop them during the data cleaning process. 

In [18]:
## drop merge indicator variable and GEO_ID
mREFI=mREFI.drop(columns=['_merge', 'GEO_ID'])
mREFI.head()

,state,fips,mrfei,fips_modified,NAME,perc_public_transport
0,AL,01001020100,16.6667,1400000US01001020100,"Census Tract 201, Autauga County, Alabama",4.651163
1,AL,01001020200,16.6667,1400000US01001020200,"Census Tract 202, Autauga County, Alabama",0.000000
2,AL,01001020300,17.6471,1400000US01001020300,"Census Tract 203, Autauga County, Alabama",0.907912
3,AL,01001020400,13.3333,1400000US01001020400,"Census Tract 204, Autauga County, Alabama",0.000000
4,AL,01001020500,6.25,1400000US01001020500,"Census Tract 205, Autauga County, Alabama",0.000000


**`ACS demographic estimates`**: the following variables will be extracted from this file: 
    1. B03002001: This is the estimate for total population. Will be used alongside land area data to generate the variable, population density. 
    2. B03002004: This is an estimate of the total number of Black or African American (alone) in the census tract. This will be used alongside B03002001 to calculate the share of the population who is Black or African American.
    3. B03001003: This is an estimate of the total number of Hispanic (alone) in the census tract. This will be used alongside B03002001 to calculate the share of the population who is Hispanic.
    4. B03002006: This is an estimate of the total number of Asian in the census tract. This will be used alongside B03002001 to calculate the share of the population who is Asian.
    5. B03002005: This is an estimate of the total number of American Indian and Alaska Native (alone) in the census tract. This will be used alongside B03002001 to calculate the share of the population who is American Indian and Alaska Native.
    6. B03002007: This is an estimate of the total number of Native Hawaiian and Other Pacific Islander (alone) in the census tract. This will be used alongside B03002001 to calculate the share of the population who is Native Hawaiian and Other Pacific Islander (alone).
    7. 

In [19]:
## import demographic data
pop_race_ethn=pd.read_csv('data/acs_5year_2010/pop_race_ethn.csv', skiprows=[1])##skip first row to remove unecessary headers
pop_race_ethn.head()

/Users/mkbelay/opt/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3071: DtypeWarning: Columns (5,25) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


,GEO_ID,NAME,B03002_001E,B03002_001M,B03002_002E,B03002_002M,B03002_003E,B03002_003M,B03002_004E,B03002_004M,...,B03002_017E,B03002_017M,B03002_018E,B03002_018M,B03002_019E,B03002_019M,B03002_020E,B03002_020M,B03002_021E,B03002_021M
0,1400000US01097002900,"Census Tract 29, Mobile County, Alabama",3622,394,3407,429,1813,280,1513,379,...,0,119,150,167,0,119,0,119,0,119
1,1400000US01071951100,"Census Tract 9511, Jackson County, Alabama",6968,519,6823,501,6519,483,15,14,...,0,119,8,12,58,61,8,15,50,61
2,1400000US01121010302,"Census Tract 103.02, Talladega County, Alabama",4303,476,4254,480,2422,495,1820,433,...,0,119,0,119,0,119,0,119,0,119
3,1400000US01097003404,"Census Tract 34.04, Mobile County, Alabama",3076,429,2988,454,418,196,2555,461,...,0,119,0,119,0,119,0,119,0,119
4,1400000US01077010800,"Census Tract 108, Lauderdale County, Alabama",3862,266,3769,228,3266,297,414,176,...,0,119,0,119,0,119,0,119,0,119


In [20]:
##keep only relevant variables
pop_race_ethn=pop_race_ethn.filter(['GEO_ID','B03002_001E', 'B03002_003E','B03002_004E','B03002_012E', 'B03002_006E', 'B03002_005E', 'B03002_007E'])
pop_race_ethn.head()


,GEO_ID,B03002_001E,B03002_003E,B03002_004E,B03002_012E,B03002_006E,B03002_005E,B03002_007E
0,1400000US01097002900,3622,1813,1513,215,0,0,81
1,1400000US01071951100,6968,6519,15,145,0,134,3
2,1400000US01121010302,4303,2422,1820,49,12,0,0
3,1400000US01097003404,3076,418,2555,88,15,0,0
4,1400000US01077010800,3862,3266,414,93,0,0,0


In [21]:
## check dtypes
pop_race_ethn.dtypes

GEO_ID         object
B03002_001E     int64
B03002_003E     int64
B03002_004E     int64
B03002_012E     int64
B03002_006E     int64
B03002_005E     int64
B03002_007E     int64
dtype: object

In [22]:
## compute perc of population who is white
pop_race_ethn['perc_white']=(pop_race_ethn['B03002_003E']/pop_race_ethn['B03002_001E'])*100
## compute perc of population who is black
pop_race_ethn['perc_black']=(pop_race_ethn['B03002_004E']/pop_race_ethn['B03002_001E'])*100
## compute perc of population who is hispanic
pop_race_ethn['perc_hispanic']=(pop_race_ethn['B03002_012E']/pop_race_ethn['B03002_001E'])*100
## compute perc of population who is asian
pop_race_ethn['perc_asian']=(pop_race_ethn['B03002_006E']/pop_race_ethn['B03002_001E'])*100
## compute perc of population who is native
pop_race_ethn['perc_native']=(pop_race_ethn['B03002_005E']/pop_race_ethn['B03002_001E'])*100
## compute perc of population who is pacific
pop_race_ethn['perc_pacific']=(pop_race_ethn['B03002_007E']/pop_race_ethn['B03002_001E'])*100

In [23]:
## check if nan values have been generated because denominator has a value of 0
pop_race_ethn.loc[pop_race_ethn.B03002_001E==0]

,GEO_ID,B03002_001E,B03002_003E,B03002_004E,B03002_012E,B03002_006E,B03002_005E,B03002_007E,perc_white,perc_black,perc_hispanic,perc_asian,perc_native,perc_pacific
140,1400000US01117980000,0,0,0,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
668,1400000US01003990000,0,0,0,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
1056,1400000US01097990000,0,0,0,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
1144,1400000US01015981903,0,0,0,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
1284,1400000US02185000300,0,0,0,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
73916,1400000US72079991100,0,0,0,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
73941,1400000US72127980103,0,0,0,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
73949,1400000US72123992800,0,0,0,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
73988,1400000US72069991800,0,0,0,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN


In [24]:
## remove observations that do not have any inhabitants [i.e, the denominator is zero]
pop_race_ethn=pop_race_ethn.loc[pop_race_ethn.B03002_001E!=0]
pop_race_ethn.head()

,GEO_ID,B03002_001E,B03002_003E,B03002_004E,B03002_012E,B03002_006E,B03002_005E,B03002_007E,perc_white,perc_black,perc_hispanic,perc_asian,perc_native,perc_pacific
0,1400000US01097002900,3622,1813,1513,215,0,0,81,50.055218,41.772501,5.935947,0.000000,0.000000,2.236334
1,1400000US01071951100,6968,6519,15,145,0,134,3,93.556257,0.215270,2.080941,0.000000,1.923077,0.043054
2,1400000US01121010302,4303,2422,1820,49,12,0,0,56.286312,42.296073,1.138740,0.278875,0.000000,0.000000
3,1400000US01097003404,3076,418,2555,88,15,0,0,13.589077,83.062419,2.860858,0.487646,0.000000,0.000000
4,1400000US01077010800,3862,3266,414,93,0,0,0,84.567582,10.719834,2.408079,0.000000,0.000000,0.000000


In [25]:
## keep only relevant variables
pop_race_ethn=pop_race_ethn.filter(['GEO_ID', 'perc_white','perc_black', 'perc_hispanic', 'perc_asian', 'perc_native', 'perc_pacific'])
pop_race_ethn.head()

,GEO_ID,perc_white,perc_black,perc_hispanic,perc_asian,perc_native,perc_pacific
0,1400000US01097002900,50.055218,41.772501,5.935947,0.000000,0.000000,2.236334
1,1400000US01071951100,93.556257,0.215270,2.080941,0.000000,1.923077,0.043054
2,1400000US01121010302,56.286312,42.296073,1.138740,0.278875,0.000000,0.000000
3,1400000US01097003404,13.589077,83.062419,2.860858,0.487646,0.000000,0.000000
4,1400000US01077010800,84.567582,10.719834,2.408079,0.000000,0.000000,0.000000


In [26]:
## merge  variables onot the mREFI dataframe
mREFI=pd.merge(left=mREFI, right=pop_race_ethn, how='left', left_on='fips_modified', right_on='GEO_ID', indicator=True)

In [27]:
## check left-only merges
mREFI.loc[mREFI['_merge']=='left_only']

,state,fips,mrfei,fips_modified,NAME,perc_public_transport,GEO_ID,perc_white,perc_black,perc_hispanic,perc_asian,perc_native,perc_pacific,_merge
7,AL,01001020800,0,1400000US01001020800,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,left_only
22,AL,01003010901,41.1765,1400000US01003010901,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,left_only
23,AL,01003010902,18.1818,1400000US01003010902,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,left_only
25,AL,01003011100,16.6667,1400000US01003011100,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,left_only
31,AL,01003011404,15.7895,1400000US01003011404,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,left_only
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
65312,WY,56029995500,12.5,1400000US56029995500,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,left_only
65321,WY,56035000100,22.2222,1400000US56035000100,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,left_only
65334,WY,56039997600,25,1400000US56039997600,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,left_only
65335,WY,56039997700,10,1400000US56039997700,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,left_only


In [28]:
## drop merge indicator variable and GEO_ID
mREFI=mREFI.drop(columns=['_merge', 'GEO_ID'])
mREFI.head()

,state,fips,mrfei,fips_modified,NAME,perc_public_transport,perc_white,perc_black,perc_hispanic,perc_asian,perc_native,perc_pacific
0,AL,01001020100,16.6667,1400000US01001020100,"Census Tract 201, Autauga County, Alabama",4.651163,78.717523,16.196794,0.829187,0.000000,0.000000,0.0
1,AL,01001020200,16.6667,1400000US01001020200,"Census Tract 202, Autauga County, Alabama",0.000000,38.465347,59.158416,0.297030,2.079208,0.000000,0.0
2,AL,01001020300,17.6471,1400000US01001020300,"Census Tract 203, Autauga County, Alabama",0.907912,82.020886,16.596105,1.100762,0.282247,0.000000,0.0
3,AL,01001020400,13.3333,1400000US01001020400,"Census Tract 204, Autauga County, Alabama",0.000000,93.863636,2.314050,2.644628,0.392562,0.000000,0.0
4,AL,01001020500,6.25,1400000US01001020500,"Census Tract 205, Autauga County, Alabama",0.000000,80.177098,11.581807,4.739384,1.730731,0.201248,0.0


**`Median Household Income`**

In [29]:
##import data
median_hh_income=pd.read_csv('data/acs_5year_2010/median_hh_income.csv', skiprows=[1])
median_hh_income.head()

,GEO_ID,NAME,B19013_001E,B19013_001M
0,1400000US01003990000,"Census Tract 9900, Baldwin County, Alabama",-,**
1,1400000US01015981901,"Census Tract 9819.01, Calhoun County, Alabama",-,**
2,1400000US01015981902,"Census Tract 9819.02, Calhoun County, Alabama",-,**
3,1400000US01015981903,"Census Tract 9819.03, Calhoun County, Alabama",-,**
4,1400000US01097003605,"Census Tract 36.05, Mobile County, Alabama",-,**


In [30]:
## keep only relevant variables
median_hh_income=median_hh_income.filter(['GEO_ID', 'B19013_001E'])

In [31]:
## check dtypes
median_hh_income.dtypes

GEO_ID         object
B19013_001E    object
dtype: object

In [32]:
## convert median income to numeric
median_hh_income['B19013_001E']=pd.to_numeric(median_hh_income['B19013_001E'], errors='coerce')#coerce to convert blank enteries to Nan

In [33]:
## rename varaible
median_hh_income=median_hh_income.rename(columns={'B19013_001E': 'median_income'})

In [34]:
## merge  variables onto the mREFI dataframe
mREFI=pd.merge(left=mREFI, right=median_hh_income, how='left', left_on='fips_modified', right_on='GEO_ID', indicator=True)

In [35]:
## check left-only merges
mREFI.loc[mREFI['_merge']=='left_only']

,state,fips,mrfei,fips_modified,NAME,perc_public_transport,perc_white,perc_black,perc_hispanic,perc_asian,perc_native,perc_pacific,GEO_ID,median_income,_merge
7,AL,01001020800,0,1400000US01001020800,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,left_only
22,AL,01003010901,41.1765,1400000US01003010901,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,left_only
23,AL,01003010902,18.1818,1400000US01003010902,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,left_only
25,AL,01003011100,16.6667,1400000US01003011100,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,left_only
31,AL,01003011404,15.7895,1400000US01003011404,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,left_only
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
65312,WY,56029995500,12.5,1400000US56029995500,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,left_only
65321,WY,56035000100,22.2222,1400000US56035000100,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,left_only
65334,WY,56039997600,25,1400000US56039997600,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,left_only
65335,WY,56039997700,10,1400000US56039997700,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,left_only


In [36]:
## drop merge indicator variable and GEO_ID
mREFI=mREFI.drop(columns=['_merge', 'GEO_ID'])
mREFI.head()

,state,fips,mrfei,fips_modified,NAME,perc_public_transport,perc_white,perc_black,perc_hispanic,perc_asian,perc_native,perc_pacific,median_income
0,AL,01001020100,16.6667,1400000US01001020100,"Census Tract 201, Autauga County, Alabama",4.651163,78.717523,16.196794,0.829187,0.000000,0.000000,0.0,70222.0
1,AL,01001020200,16.6667,1400000US01001020200,"Census Tract 202, Autauga County, Alabama",0.000000,38.465347,59.158416,0.297030,2.079208,0.000000,0.0,41091.0
2,AL,01001020300,17.6471,1400000US01001020300,"Census Tract 203, Autauga County, Alabama",0.907912,82.020886,16.596105,1.100762,0.282247,0.000000,0.0,44031.0
3,AL,01001020400,13.3333,1400000US01001020400,"Census Tract 204, Autauga County, Alabama",0.000000,93.863636,2.314050,2.644628,0.392562,0.000000,0.0,56627.0
4,AL,01001020500,6.25,1400000US01001020500,"Census Tract 205, Autauga County, Alabama",0.000000,80.177098,11.581807,4.739384,1.730731,0.201248,0.0,68317.0


**`Populaion density`**: this attribute will be generated by dividing the total population in a census tract by the land area (SQMI) of that census tract

In [37]:
## import census tract land area data
land_area=pd.read_csv('data/USA_Census_Tract_Areas.csv')
land_area

,OBJECTID,FIPS,COUNTY,STATE,POPULATION,SQMI,Shape__Area,Shape__Length
0,1,15007041200,Kauai,HI,190.0,72.30,2.185558e+08,1.055210e+05
1,2,15007990200,Kauai,HI,0.0,0.07,1.974252e+05,3.050482e+03
2,3,15007040604,Kauai,HI,3549.0,8.59,2.596760e+07,2.780688e+04
3,4,15007040700,Kauai,HI,8951.0,36.55,1.105660e+08,7.255392e+04
4,5,15007040800,Kauai,HI,3965.0,120.86,3.660431e+08,9.958471e+04
...,...,...,...,...,...,...,...,...
73677,73678,23003951900,Aroostook,ME,2793.0,20.01,1.100000e+08,5.352778e+04
73678,73679,23003952000,Aroostook,ME,2918.0,24.80,1.361858e+08,6.211481e+04
73679,73680,23003952100,Aroostook,ME,2690.0,94.08,5.148271e+08,1.005318e+05
73680,73681,23003952300,Aroostook,ME,2713.0,720.54,3.920575e+09,3.509525e+05


In [38]:
## check data types
land_area.dtypes

OBJECTID           int64
FIPS               int64
COUNTY            object
STATE             object
POPULATION       float64
SQMI             float64
Shape__Area      float64
Shape__Length    float64
dtype: object

Similar to the mRFEI Index, the land area data from census does not have the leading string that accompanies all FIPS codes in the ACS data. This string will be added to all FIPS codes in the land area data set.

In [39]:
## convert FIPS column to string
land_area.FIPS=land_area.FIPS.astype(str)

In [40]:
## add census tract FIPS identifier to each FIPS
land_area['FIPS_mod']=land_area[land_area['FIPS'].str.len()> 10]['FIPS'].apply(lambda x: '1400000US' + str(x))

In [41]:
## add census tract FIPS identifier to each FIPS
land_area['FIPS_mod_2']=land_area[land_area['FIPS'].str.len()==10]['FIPS'].apply(lambda x: '1400000US0' + str(x))

In [42]:
## combine revised FIPS codes into single column
land_area.FIPS_mod=land_area.FIPS_mod.fillna(land_area.FIPS_mod_2)

In [43]:
## keep only relevant columns
land_area=land_area.filter(['COUNTY', 'SQMI', 'FIPS_mod'])
land_area

,COUNTY,SQMI,FIPS_mod
0,Kauai,72.30,1400000US15007041200
1,Kauai,0.07,1400000US15007990200
2,Kauai,8.59,1400000US15007040604
3,Kauai,36.55,1400000US15007040700
4,Kauai,120.86,1400000US15007040800
...,...,...,...
73677,Aroostook,20.01,1400000US23003951900
73678,Aroostook,24.80,1400000US23003952000
73679,Aroostook,94.08,1400000US23003952100
73680,Aroostook,720.54,1400000US23003952300


In [44]:
## compute population density

##----1)import total population variable 
total_pop=pd.read_csv('data/acs_5year_2010/pop_race_ethn.csv', skiprows=[1])## import population data set
total_pop=total_pop.filter(['GEO_ID', 'B03002_001E'])## keep only variable that represents total population
total_pop.head()

/Users/mkbelay/opt/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3071: DtypeWarning: Columns (5,25) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


,GEO_ID,B03002_001E
0,1400000US01097002900,3622
1,1400000US01071951100,6968
2,1400000US01121010302,4303
3,1400000US01097003404,3076
4,1400000US01077010800,3862


In [45]:
##-----2) merge onto landarea datafrmae
land_area=pd.merge(left=land_area, right=total_pop, how='left', left_on='FIPS_mod', right_on='GEO_ID', indicator=True)

In [46]:
## check left_only merges
land_area[land_area['_merge']=='left_only']

,COUNTY,SQMI,FIPS_mod,GEO_ID,B03002_001E,_merge
336,Kusilvak,17969.67,1400000US02158000100,NaN,NaN,left_only
21339,Oglala Lakota,161.55,1400000US46102940500,NaN,NaN,left_only
21340,Oglala Lakota,1245.53,1400000US46102940800,NaN,NaN,left_only
21341,Oglala Lakota,689.49,1400000US46102940900,NaN,NaN,left_only
50935,Bedford County,6.90,1400000US51019050100,NaN,NaN,left_only


In [47]:
## check value counts for merge indicator
land_area._merge.value_counts()

both          73677
left_only         5
right_only        0
Name: _merge, dtype: int64

There are census tracts in the land area data set that are not present in the population data set and thus have an NA for total population. These will be removed. 

In [48]:
#-----------2.1) remove census tracts that are missing values for total population
land_area=land_area.loc[land_area['_merge']=='both']
land_area.head()

,COUNTY,SQMI,FIPS_mod,GEO_ID,B03002_001E,_merge
0,Kauai,72.30,1400000US15007041200,1400000US15007041200,0.0,both
1,Kauai,0.07,1400000US15007990200,1400000US15007990200,0.0,both
2,Kauai,8.59,1400000US15007040604,1400000US15007040604,3036.0,both
3,Kauai,36.55,1400000US15007040700,1400000US15007040700,8409.0,both
4,Kauai,120.86,1400000US15007040800,1400000US15007040800,3845.0,both


In [49]:
##-------3) compute population density
land_area['pop_density']=land_area['B03002_001E']/land_area['SQMI']
land_area

,COUNTY,SQMI,FIPS_mod,GEO_ID,B03002_001E,_merge,pop_density
0,Kauai,72.30,1400000US15007041200,1400000US15007041200,0.0,both,0.000000
1,Kauai,0.07,1400000US15007990200,1400000US15007990200,0.0,both,0.000000
2,Kauai,8.59,1400000US15007040604,1400000US15007040604,3036.0,both,353.434226
3,Kauai,36.55,1400000US15007040700,1400000US15007040700,8409.0,both,230.068399
4,Kauai,120.86,1400000US15007040800,1400000US15007040800,3845.0,both,31.813669
...,...,...,...,...,...,...,...
73677,Aroostook,20.01,1400000US23003951900,1400000US23003951900,2741.0,both,136.981509
73678,Aroostook,24.80,1400000US23003952000,1400000US23003952000,3255.0,both,131.250000
73679,Aroostook,94.08,1400000US23003952100,1400000US23003952100,2812.0,both,29.889456
73680,Aroostook,720.54,1400000US23003952300,1400000US23003952300,2708.0,both,3.758292


In [50]:
##----check for denominators with a value of zero
land_area[(land_area['SQMI']==0)]

,COUNTY,SQMI,FIPS_mod,GEO_ID,B03002_001E,_merge,pop_density
7432,Curry,0.0,1400000US41015990101,1400000US41015990101,0.0,both,NaN
44171,NaN,0.0,1400000US72023990000,1400000US72023990000,0.0,both,NaN
44853,NaN,0.0,1400000US72033990201,1400000US72033990201,0.0,both,NaN
51565,Hampton,0.0,1400000US51650990100,1400000US51650990100,0.0,both,NaN
51854,Northumberland,0.0,1400000US51133990100,1400000US51133990100,0.0,both,NaN


Census tracts with a SQMI of zero will be removed from the data set.

In [51]:
##----- remove census tracts with SQMI of zero
land_area=land_area[(land_area['SQMI']!=0)]

In [52]:
##remove census tracts that are uninhabited
land_area=land_area[(land_area['B03002_001E']!=0)]
land_area.head()

,COUNTY,SQMI,FIPS_mod,GEO_ID,B03002_001E,_merge,pop_density
2,Kauai,8.59,1400000US15007040604,1400000US15007040604,3036.0,both,353.434226
3,Kauai,36.55,1400000US15007040700,1400000US15007040700,8409.0,both,230.068399
4,Kauai,120.86,1400000US15007040800,1400000US15007040800,3845.0,both,31.813669
5,Kauai,89.66,1400000US15007040900,1400000US15007040900,4679.0,both,52.186036
6,Kauai,79.92,1400000US15007040104,1400000US15007040104,1083.0,both,13.551051


In [53]:
##---keep only relevant variables
land_area=land_area.filter(['GEO_ID', 'SQMI', 'pop_density'])

In [54]:
##--- merge onto main dataframe 
mREFI=pd.merge(left=mREFI, right=land_area, how='left', left_on='fips_modified', right_on='GEO_ID', indicator=True)


In [55]:
## check left_only merges
mREFI[mREFI['_merge']=='left_only']

,state,fips,mrfei,fips_modified,NAME,perc_public_transport,perc_white,perc_black,perc_hispanic,perc_asian,perc_native,perc_pacific,median_income,GEO_ID,SQMI,pop_density,_merge
7,AL,01001020800,0,1400000US01001020800,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,left_only
22,AL,01003010901,41.1765,1400000US01003010901,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,left_only
23,AL,01003010902,18.1818,1400000US01003010902,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,left_only
25,AL,01003011100,16.6667,1400000US01003011100,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,left_only
31,AL,01003011404,15.7895,1400000US01003011404,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,left_only
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
65312,WY,56029995500,12.5,1400000US56029995500,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,left_only
65321,WY,56035000100,22.2222,1400000US56035000100,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,left_only
65334,WY,56039997600,25,1400000US56039997600,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,left_only
65335,WY,56039997700,10,1400000US56039997700,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,left_only


In [56]:
## drop merge indicator variable
mREFI=mREFI.drop(columns=['_merge', 'GEO_ID'])
mREFI.head()

,state,fips,mrfei,fips_modified,NAME,perc_public_transport,perc_white,perc_black,perc_hispanic,perc_asian,perc_native,perc_pacific,median_income,SQMI,pop_density
0,AL,01001020100,16.6667,1400000US01001020100,"Census Tract 201, Autauga County, Alabama",4.651163,78.717523,16.196794,0.829187,0.000000,0.000000,0.0,70222.0,3.78,478.571429
1,AL,01001020200,16.6667,1400000US01001020200,"Census Tract 202, Autauga County, Alabama",0.000000,38.465347,59.158416,0.297030,2.079208,0.000000,0.0,41091.0,1.29,1565.891473
2,AL,01001020300,17.6471,1400000US01001020300,"Census Tract 203, Autauga County, Alabama",0.907912,82.020886,16.596105,1.100762,0.282247,0.000000,0.0,44031.0,2.07,1711.594203
3,AL,01001020400,13.3333,1400000US01001020400,"Census Tract 204, Autauga County, Alabama",0.000000,93.863636,2.314050,2.644628,0.392562,0.000000,0.0,56627.0,2.45,1975.510204
4,AL,01001020500,6.25,1400000US01001020500,"Census Tract 205, Autauga County, Alabama",0.000000,80.177098,11.581807,4.739384,1.730731,0.201248,0.0,68317.0,4.42,2248.416290


**`Poverty rate`**: this will represent the share of population that is below the poverty line. This attribute will be generated by dividing the number of people below the poverty line by the total number of people in the census tract

In [57]:
pov=pd.read_csv('data/acs_5year_2010/poverty.csv')
pov.head()

/Users/mkbelay/opt/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3071: DtypeWarning: Columns (2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


,GEO_ID,NAME,B17001_001E,B17001_001M,B17001_002E,B17001_002M,B17001_003E,B17001_003M,B17001_004E,B17001_004M,...,B17001_055E,B17001_055M,B17001_056E,B17001_056M,B17001_057E,B17001_057M,B17001_058E,B17001_058M,B17001_059E,B17001_059M
0,id,Geographic Area Name,Estimate!!Total,Margin of Error!!Total,Estimate!!Total!!Income in the past 12 months ...,Margin of Error!!Total!!Income in the past 12 ...,Estimate!!Total!!Income in the past 12 months ...,Margin of Error!!Total!!Income in the past 12 ...,Estimate!!Total!!Income in the past 12 months ...,Margin of Error!!Total!!Income in the past 12 ...,...,Estimate!!Total!!Income in the past 12 months ...,Margin of Error!!Total!!Income in the past 12 ...,Estimate!!Total!!Income in the past 12 months ...,Margin of Error!!Total!!Income in the past 12 ...,Estimate!!Total!!Income in the past 12 months ...,Margin of Error!!Total!!Income in the past 12 ...,Estimate!!Total!!Income in the past 12 months ...,Margin of Error!!Total!!Income in the past 12 ...,Estimate!!Total!!Income in the past 12 months ...,Margin of Error!!Total!!Income in the past 12 ...
1,1400000US01001020100,"Census Tract 201, Autauga County, Alabama",1809,155,190,97,109,63,30,37,...,129,41,182,51,87,50,60,27,30,28
2,1400000US01001020200,"Census Tract 202, Autauga County, Alabama",1992,243,294,169,171,96,20,30,...,243,96,116,53,69,38,56,36,79,50
3,1400000US01001020300,"Census Tract 203, Autauga County, Alabama",3533,328,284,198,71,79,0,119,...,194,73,212,85,196,68,133,54,150,69
4,1400000US01001020400,"Census Tract 204, Autauga County, Alabama",4840,260,306,106,131,58,24,23,...,317,49,331,48,285,45,305,49,229,49


In [58]:
## keep only relevant variables
pov=pov.filter(['B17001_002E','B17001_001E', 'GEO_ID'])##number with income below poverty; total population for whom poverty status is determined.
pov.head()

,B17001_002E,B17001_001E,GEO_ID
0,Estimate!!Total!!Income in the past 12 months ...,Estimate!!Total,id
1,190,1809,1400000US01001020100
2,294,1992,1400000US01001020200
3,284,3533,1400000US01001020300
4,306,4840,1400000US01001020400


In [59]:
## remove first row; unecessary header
pov=pov.loc[1:,:]
pov.head()

,B17001_002E,B17001_001E,GEO_ID
1,190,1809,1400000US01001020100
2,294,1992,1400000US01001020200
3,284,3533,1400000US01001020300
4,306,4840,1400000US01001020400
5,592,9938,1400000US01001020500


In [60]:
## check dtypes
pov.dtypes

B17001_002E    object
B17001_001E    object
GEO_ID         object
dtype: object

In [61]:
## change dtypes to numeric 
pov['B17001_002E'] = pd.to_numeric(pov['B17001_002E'])
pov['B17001_001E'] = pd.to_numeric(pov['B17001_001E'])

In [62]:
## compute share of population below poverty line
pov['perc_pov']=(pov['B17001_002E']/pov['B17001_001E'])*100
pov.head()

,B17001_002E,B17001_001E,GEO_ID,perc_pov
1,190,1809,1400000US01001020100,10.503040
2,294,1992,1400000US01001020200,14.759036
3,284,3533,1400000US01001020300,8.038494
4,306,4840,1400000US01001020400,6.322314
5,592,9938,1400000US01001020500,5.956933


In [63]:
## drop observations if the denominator (i.e, total population) is zero
pov=pov.loc[pov['B17001_001E']!=0]
pov.head()

,B17001_002E,B17001_001E,GEO_ID,perc_pov
1,190,1809,1400000US01001020100,10.503040
2,294,1992,1400000US01001020200,14.759036
3,284,3533,1400000US01001020300,8.038494
4,306,4840,1400000US01001020400,6.322314
5,592,9938,1400000US01001020500,5.956933


In [64]:
## keep only relevant variables
pov=pov.filter(['GEO_ID','perc_pov'])
pov

,GEO_ID,perc_pov
1,1400000US01001020100,10.503040
2,1400000US01001020200,14.759036
3,1400000US01001020300,8.038494
4,1400000US01001020400,6.322314
5,1400000US01001020500,5.956933
...,...,...
73998,1400000US72127000900,14.606328
73999,1400000US72127001000,21.914304
74000,1400000US72127001100,8.887508
74001,1400000US72127001200,15.843977


In [65]:
## merge onto main dataframe
mREFI=pd.merge(left=mREFI, right=pov, how='left', left_on='fips_modified', right_on='GEO_ID', indicator=True)


In [66]:
##check left-only merge
mREFI[mREFI._merge=='left_only']

,state,fips,mrfei,fips_modified,NAME,perc_public_transport,perc_white,perc_black,perc_hispanic,perc_asian,perc_native,perc_pacific,median_income,SQMI,pop_density,GEO_ID,perc_pov,_merge
7,AL,01001020800,0,1400000US01001020800,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,left_only
22,AL,01003010901,41.1765,1400000US01003010901,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,left_only
23,AL,01003010902,18.1818,1400000US01003010902,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,left_only
25,AL,01003011100,16.6667,1400000US01003011100,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,left_only
31,AL,01003011404,15.7895,1400000US01003011404,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,left_only
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
65312,WY,56029995500,12.5,1400000US56029995500,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,left_only
65321,WY,56035000100,22.2222,1400000US56035000100,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,left_only
65334,WY,56039997600,25,1400000US56039997600,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,left_only
65335,WY,56039997700,10,1400000US56039997700,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,left_only


In [67]:
## drop merge indicator variable 
mREFI=mREFI.drop(columns=['_merge', 'GEO_ID'])

**`Households with SNAP`** :this will represent the share of population that is on SNAP. This attribute will be generated by dividing the number of people on SNAP by the total number of people in the census tract

In [68]:
## import data
snap=pd.read_csv('data/acs_5year_2010/SNAP.csv')
snap.head()

/Users/mkbelay/opt/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3071: DtypeWarning: Columns (2,3,4,5,6,7) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


,GEO_ID,NAME,B19058_001E,B19058_001M,B19058_002E,B19058_002M,B19058_003E,B19058_003M
0,id,Geographic Area Name,Estimate!!Total,Margin of Error!!Total,Estimate!!Total!!With cash public assistance o...,Margin of Error!!Total!!With cash public assis...,Estimate!!Total!!No cash public assistance or ...,Margin of Error!!Total!!No cash public assista...
1,1400000US01001020100,"Census Tract 201, Autauga County, Alabama",696,72,40,37,656,69
2,1400000US01001020200,"Census Tract 202, Autauga County, Alabama",730,85,65,47,665,84
3,1400000US01001020300,"Census Tract 203, Autauga County, Alabama",1287,110,125,76,1162,123
4,1400000US01001020400,"Census Tract 204, Autauga County, Alabama",1839,94,123,45,1716,90


In [69]:
## keep relevant variables
snap=snap.filter(['B19058_001E', 'B19058_002E', 'GEO_ID'])

In [70]:
## remove first row
snap=snap.loc[1:,:]

In [71]:
## convert variables to numeric
snap['B19058_001E']=pd.to_numeric(snap['B19058_001E'])
snap['B19058_002E']=pd.to_numeric(snap['B19058_002E'])

In [72]:
## compute share of population on SNAP
snap['perc_snap']=(snap['B19058_002E']/snap['B19058_001E'])*100
snap.head()

,B19058_001E,B19058_002E,GEO_ID,perc_snap
1,696,40,1400000US01001020100,5.747126
2,730,65,1400000US01001020200,8.904110
3,1287,125,1400000US01001020300,9.712510
4,1839,123,1400000US01001020400,6.688418
5,3741,140,1400000US01001020500,3.742315


In [73]:
## check if denominator is equal to 0 and if perc_snap takes on a Nan value because of that
snap.loc[snap['B19058_001E']==0]

,B19058_001E,B19058_002E,GEO_ID,perc_snap
334,0,0,1400000US01003990000,NaN
399,0,0,1400000US01015981903,NaN
806,0,0,1400000US01097003605,NaN
869,0,0,1400000US01097990000,NaN
1064,0,0,1400000US01117980000,NaN
...,...,...,...,...
73951,0,0,1400000US72113993000,NaN
73956,0,0,1400000US72115990000,NaN
73960,0,0,1400000US72117990400,NaN
73972,0,0,1400000US72119992700,NaN


In [74]:
## remove observations with a zero value for the denominator
snap=snap.loc[snap['B19058_001E']!=0]
snap.head()

,B19058_001E,B19058_002E,GEO_ID,perc_snap
1,696,40,1400000US01001020100,5.747126
2,730,65,1400000US01001020200,8.904110
3,1287,125,1400000US01001020300,9.712510
4,1839,123,1400000US01001020400,6.688418
5,3741,140,1400000US01001020500,3.742315


In [75]:
# keep relevant variables 
snap=snap.filter(['GEO_ID', 'perc_snap'])

In [76]:
## merge onto main dataframe
mREFI=pd.merge(left=mREFI, right=snap, how='left', left_on='fips_modified', right_on='GEO_ID', indicator=True)


In [77]:
##check left-only merge
mREFI[mREFI._merge=='left_only']

,state,fips,mrfei,fips_modified,NAME,perc_public_transport,perc_white,perc_black,perc_hispanic,perc_asian,perc_native,perc_pacific,median_income,SQMI,pop_density,perc_pov,GEO_ID,perc_snap,_merge
7,AL,01001020800,0,1400000US01001020800,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,left_only
22,AL,01003010901,41.1765,1400000US01003010901,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,left_only
23,AL,01003010902,18.1818,1400000US01003010902,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,left_only
25,AL,01003011100,16.6667,1400000US01003011100,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,left_only
31,AL,01003011404,15.7895,1400000US01003011404,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,left_only
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
65312,WY,56029995500,12.5,1400000US56029995500,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,left_only
65321,WY,56035000100,22.2222,1400000US56035000100,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,left_only
65334,WY,56039997600,25,1400000US56039997600,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,left_only
65335,WY,56039997700,10,1400000US56039997700,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,left_only


In [78]:
## remove  merge indicator variable and GEO_ID variable 
mREFI=mREFI.drop(columns=['GEO_ID', '_merge'])

**`Inequality`**

In [79]:
## import data
gini=snap=pd.read_csv('data/acs_5year_2010/gini.csv')
gini.head()

,GEO_ID,NAME,B19083_001E,B19083_001M
0,id,Geographic Area Name,Estimate!!Gini Index,Margin of Error!!Gini Index
1,1400000US01003990000,"Census Tract 9900, Baldwin County, Alabama",-,**
2,1400000US01015981901,"Census Tract 9819.01, Calhoun County, Alabama",-,**
3,1400000US01015981902,"Census Tract 9819.02, Calhoun County, Alabama",-,**
4,1400000US01015981903,"Census Tract 9819.03, Calhoun County, Alabama",-,**


In [80]:
## check dtypes
gini.dtypes

GEO_ID         object
NAME           object
B19083_001E    object
B19083_001M    object
dtype: object

In [81]:
## remove first row
gini=gini.loc[1:,:]

In [82]:
## convert gini index variable to numeric
gini['B19083_001E']=pd.to_numeric(gini['B19083_001E'], errors='coerce')

<ipython-input-82-814e51003b1b>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  gini['B19083_001E']=pd.to_numeric(gini['B19083_001E'], errors='coerce')


In [83]:
## confirm that conversion worked
gini.dtypes

GEO_ID          object
NAME            object
B19083_001E    float64
B19083_001M     object
dtype: object

In [84]:
## keep only relevant vars
gini=gini.filter(['GEO_ID','B19083_001E'])

In [85]:
## merge onto main dataframe
mREFI=pd.merge(left=mREFI, right=gini, how='left', left_on='fips_modified', right_on='GEO_ID', indicator=True)


In [86]:
##check left-only merge
mREFI[mREFI._merge=='left_only']

,state,fips,mrfei,fips_modified,NAME,perc_public_transport,perc_white,perc_black,perc_hispanic,perc_asian,perc_native,perc_pacific,median_income,SQMI,pop_density,perc_pov,perc_snap,GEO_ID,B19083_001E,_merge
7,AL,01001020800,0,1400000US01001020800,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,left_only
22,AL,01003010901,41.1765,1400000US01003010901,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,left_only
23,AL,01003010902,18.1818,1400000US01003010902,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,left_only
25,AL,01003011100,16.6667,1400000US01003011100,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,left_only
31,AL,01003011404,15.7895,1400000US01003011404,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,left_only
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
65312,WY,56029995500,12.5,1400000US56029995500,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,left_only
65321,WY,56035000100,22.2222,1400000US56035000100,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,left_only
65334,WY,56039997600,25,1400000US56039997600,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,left_only
65335,WY,56039997700,10,1400000US56039997700,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,left_only


In [87]:
## remove  merge indicator variable and GEO_ID variable 
mREFI=mREFI.drop(columns=['GEO_ID', '_merge'])

In [88]:
## reame gini index variable
mREFI=mREFI.rename(columns={'B19083_001E':'inequality'})

**`Unemployment`**

In [89]:
unemploy=pd.read_csv('data/acs_5year_2010/employment.csv')
unemploy.head()

/Users/mkbelay/opt/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3071: DtypeWarning: Columns (2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,50,51,52,53,54,55,56,57,58,59) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


,GEO_ID,NAME,S2301_C01_001E,S2301_C01_001M,S2301_C01_002E,S2301_C01_002M,S2301_C01_003E,S2301_C01_003M,S2301_C01_004E,S2301_C01_004M,...,S2301_C04_026E,S2301_C04_026M,S2301_C04_027E,S2301_C04_027M,S2301_C04_028E,S2301_C04_028M,S2301_C04_029E,S2301_C04_029M,S2301_C04_030E,S2301_C04_030M
0,id,Geographic Area Name,Total!!Estimate!!Population 16 years and over,Total!!Margin of Error!!Population 16 years an...,Total!!Estimate!!AGE!!16 to 19 years,Total!!Margin of Error!!AGE!!16 to 19 years,Total!!Estimate!!AGE!!20 to 24 years,Total!!Margin of Error!!AGE!!20 to 24 years,Total!!Estimate!!AGE!!25 to 44 years,Total!!Margin of Error!!AGE!!25 to 44 years,...,Unemployment rate!!Estimate!!EDUCATIONAL ATTAI...,Unemployment rate!!Margin of Error!!EDUCATIONA...,Unemployment rate!!Estimate!!EDUCATIONAL ATTAI...,Unemployment rate!!Margin of Error!!EDUCATIONA...,Unemployment rate!!Estimate!!EDUCATIONAL ATTAI...,Unemployment rate!!Margin of Error!!EDUCATIONA...,Unemployment rate!!Estimate!!EDUCATIONAL ATTAI...,Unemployment rate!!Margin of Error!!EDUCATIONA...,Unemployment rate!!Estimate!!PERCENT IMPUTED!!...,Unemployment rate!!Margin of Error!!PERCENT IM...
1,1400000US01055011001,"Census Tract 110.01, Etowah County, Alabama",3440,232,274,96,209,52,1079,111,...,18.9,22.4,1.5,1.6,5.0,4.6,1.0,1.7,(X),(X)
2,1400000US01055011002,"Census Tract 110.02, Etowah County, Alabama",3827,341,294,166,299,127,1136,217,...,20.5,22.7,0.0,6.1,1.5,2.9,0.0,14.5,(X),(X)
3,1400000US01055011100,"Census Tract 111, Etowah County, Alabama",3739,308,172,77,321,127,1336,167,...,6.1,6.8,2.6,2.8,1.9,2.0,3.9,6.8,(X),(X)
4,1400000US01055011200,"Census Tract 112, Etowah County, Alabama",1948,368,213,117,268,145,736,255,...,9.5,12.5,45.5,29.2,9.2,10.5,0.0,36.7,(X),(X)


In [90]:
## keep only relevant vars
unemploy=unemploy.filter(['GEO_ID','S2301_C04_001E'])

In [91]:
## check dtypes
unemploy['S2301_C04_001E'].dtype## unemployment rate variable

dtype('O')

In [92]:
## remove first row
unemploy=unemploy.loc[1:,:]

In [93]:
## convert to numeric
unemploy['S2301_C04_001E']=pd.to_numeric(unemploy['S2301_C04_001E'], errors='coerce')

In [94]:
## rename variable 
unemploy=unemploy.rename(columns={'S2301_C04_001E':'unemploy'})

In [95]:
## merge onto main dataframe
mREFI=pd.merge(left=mREFI, right=unemploy, how='left', left_on='fips_modified', right_on='GEO_ID', indicator=True)


In [96]:
##check left-only merge
mREFI[mREFI._merge=='left_only']## merge is successful

,state,fips,mrfei,fips_modified,NAME,perc_public_transport,perc_white,perc_black,perc_hispanic,perc_asian,...,perc_pacific,median_income,SQMI,pop_density,perc_pov,perc_snap,inequality,GEO_ID,unemploy,_merge
7,AL,01001020800,0,1400000US01001020800,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,left_only
22,AL,01003010901,41.1765,1400000US01003010901,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,left_only
23,AL,01003010902,18.1818,1400000US01003010902,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,left_only
25,AL,01003011100,16.6667,1400000US01003011100,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,left_only
31,AL,01003011404,15.7895,1400000US01003011404,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,left_only
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
65312,WY,56029995500,12.5,1400000US56029995500,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,left_only
65321,WY,56035000100,22.2222,1400000US56035000100,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,left_only
65334,WY,56039997600,25,1400000US56039997600,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,left_only
65335,WY,56039997700,10,1400000US56039997700,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,left_only


In [97]:
## remove  merge indicator variable and GEO_ID variable 
mREFI=mREFI.drop(columns=['GEO_ID', '_merge'])

**`Educational Attainment`**

In [98]:
## import data
education=pd.read_csv('data/acs_5year_2010/educ_attainment.csv')
education.head()

/Users/mkbelay/opt/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3071: DtypeWarning: Columns (2,3,12,13,32,33,38,39,44,45,50,51,78,79,88,89,108,109,114,115,120,121,126,127,152,155,164,165,184,185,190,191,196,197,202,203) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


,GEO_ID,NAME,S1501_C01_001E,S1501_C01_001M,S1501_C01_002E,S1501_C01_002M,S1501_C01_003E,S1501_C01_003M,S1501_C01_004E,S1501_C01_004M,...,S1501_C03_034E,S1501_C03_034M,S1501_C03_035E,S1501_C03_035M,S1501_C03_036E,S1501_C03_036M,S1501_C03_037E,S1501_C03_037M,S1501_C03_038E,S1501_C03_038M
0,id,Geographic Area Name,Total!!Estimate!!Population 18 to 24 years,Total!!Margin of Error!!Population 18 to 24 years,Total!!Estimate!!Less than high school graduate,Total!!Margin of Error!!Less than high school ...,Total!!Estimate!!High school graduate (include...,Total!!Margin of Error!!High school graduate (...,Total!!Estimate!!Some college or associate's d...,Total!!Margin of Error!!Some college or associ...,...,Female!!Estimate!!MEDIAN EARNINGS IN THE PAST ...,Female!!Margin of Error!!MEDIAN EARNINGS IN TH...,Female!!Estimate!!MEDIAN EARNINGS IN THE PAST ...,Female!!Margin of Error!!MEDIAN EARNINGS IN TH...,Female!!Estimate!!MEDIAN EARNINGS IN THE PAST ...,Female!!Margin of Error!!MEDIAN EARNINGS IN TH...,Female!!Estimate!!MEDIAN EARNINGS IN THE PAST ...,Female!!Margin of Error!!MEDIAN EARNINGS IN TH...,Female!!Estimate!!PERCENT IMPUTED!!Educational...,Female!!Margin of Error!!PERCENT IMPUTED!!Educ...
1,1400000US01055011001,"Census Tract 110.01, Etowah County, Alabama",311,57,11.6,9.4,38.3,15.0,32.8,13.8,...,15372,1953,21354,5902,42500,30493,49968,8372,(X),(X)
2,1400000US01055011002,"Census Tract 110.02, Etowah County, Alabama",402,142,33.8,21.5,43.0,18.1,15.9,12.7,...,20167,6949,21351,9669,31146,7920,-,**,(X),(X)
3,1400000US01055011100,"Census Tract 111, Etowah County, Alabama",392,127,31.9,18.6,23.0,12.0,38.5,23.6,...,13981,11311,16591,5825,20167,20584,43393,19038,(X),(X)
4,1400000US01055011200,"Census Tract 112, Etowah County, Alabama",369,161,27.1,18.7,42.8,23.5,30.1,17.1,...,21563,4140,9122,11245,40046,32468,-,**,(X),(X)


In [99]:
## keep only relevant vars
education=education.filter(['GEO_ID','S1501_C01_008E','S1501_C01_010E','S1501_C01_011E','S1501_C01_015E'])

In [100]:
## check dtypes of relevant variables
education.dtypes

GEO_ID            object
S1501_C01_008E    object
S1501_C01_010E    object
S1501_C01_011E    object
S1501_C01_015E    object
dtype: object

In [101]:
# remove first row 
education=education.loc[1:,:]

In [102]:
## convert to numeric
education['S1501_C01_008E']=pd.to_numeric(education['S1501_C01_008E'], errors='coerce')
education['S1501_C01_010E']=pd.to_numeric(education['S1501_C01_010E'], errors='coerce')
education['S1501_C01_011E']=pd.to_numeric(education['S1501_C01_011E'], errors='coerce')
education['S1501_C01_015E']=pd.to_numeric(education['S1501_C01_015E'], errors='coerce')

In [103]:
## check dtypes of relevant variables
education.dtypes

GEO_ID             object
S1501_C01_008E    float64
S1501_C01_010E    float64
S1501_C01_011E    float64
S1501_C01_015E    float64
dtype: object

In [104]:
## rename variables
education=education.rename(columns={'S1501_C01_008E':'below_high_school','S1501_C01_010E':'college_no_degree','S1501_C01_011E':'associates_degree','S1501_C01_015E':'bach_or_more'})

In [105]:
## merge onto main dataframe
mREFI=pd.merge(left=mREFI, right=education, how='left', left_on='fips_modified', right_on='GEO_ID', indicator=True)


In [106]:
##check left-only merge
mREFI[mREFI._merge=='left_only']

,state,fips,mrfei,fips_modified,NAME,perc_public_transport,perc_white,perc_black,perc_hispanic,perc_asian,...,perc_pov,perc_snap,inequality,unemploy,GEO_ID,below_high_school,college_no_degree,associates_degree,bach_or_more,_merge
7,AL,01001020800,0,1400000US01001020800,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,left_only
22,AL,01003010901,41.1765,1400000US01003010901,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,left_only
23,AL,01003010902,18.1818,1400000US01003010902,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,left_only
25,AL,01003011100,16.6667,1400000US01003011100,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,left_only
31,AL,01003011404,15.7895,1400000US01003011404,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,left_only
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
65312,WY,56029995500,12.5,1400000US56029995500,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,left_only
65321,WY,56035000100,22.2222,1400000US56035000100,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,left_only
65334,WY,56039997600,25,1400000US56039997600,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,left_only
65335,WY,56039997700,10,1400000US56039997700,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,left_only


In [107]:
## remove  merge indicator variable and GEO_ID variable 
mREFI=mREFI.drop(columns=['GEO_ID', '_merge'])

**`Property value`**

In [108]:
## import data
value=pd.read_csv('data/acs_5year_2010/value.csv')
value

,GEO_ID,NAME,B25077_001E,B25077_001M
0,id,Geographic Area Name,Estimate!!Median value (dollars),Margin of Error!!Median value (dollars)
1,1400000US01003990000,"Census Tract 9900, Baldwin County, Alabama",-,**
2,1400000US01015981901,"Census Tract 9819.01, Calhoun County, Alabama",-,**
3,1400000US01015981902,"Census Tract 9819.02, Calhoun County, Alabama",-,**
4,1400000US01015981903,"Census Tract 9819.03, Calhoun County, Alabama",-,**
...,...,...,...,...
73998,1400000US72054580200,"Census Tract 5802, Florida Municipio, Puerto Rico",110400,7401
73999,1400000US72007230200,"Census Tract 2302, Aguas Buenas Municipio, Pue...",93900,5055
74000,1400000US72115330400,"Census Tract 3304, Quebradillas Municipio, Pue...",101800,8933
74001,1400000US72127007900,"Census Tract 79, San Juan Municipio, Puerto Rico",216200,14231


In [109]:
value['B25077_001E'].dtypes## [median property value]

dtype('O')

In [110]:
## remove first row 
value=value.loc[1:,:]

In [111]:
## check for unusual string entries
value.loc[value['B25077_001E']=='10,000-']

,GEO_ID,NAME,B25077_001E,B25077_001M
5,1400000US01081040603,"Census Tract 406.03, Lee County, Alabama","10,000-",***
18353,1400000US13063040617,"Census Tract 406.17, Clayton County, Georgia","10,000-",***
18367,1400000US13153020600,"Census Tract 206, Houston County, Georgia","10,000-",***
27199,1400000US21111011901,"Census Tract 119.01, Jefferson County, Kentucky","10,000-",***
28321,1400000US22071006900,"Census Tract 69, Orleans Parish, Louisiana","10,000-",***
28322,1400000US22071014500,"Census Tract 145, Orleans Parish, Louisiana","10,000-",***
32738,1400000US26077001507,"Census Tract 15.07, Kalamazoo County, Michigan","10,000-",***
39714,1400000US32031001008,"Census Tract 10.08, Washoe County, Nevada","10,000-",***
43221,1400000US36005020100,"Census Tract 201, Bronx County, New York","10,000-",***
43367,1400000US36061000600,"Census Tract 6, New York County, New York","10,000-",***


In [112]:
## check for unusual string entries
value.loc[value['B25077_001E']=='1,000,000+']

,GEO_ID,NAME,B25077_001E,B25077_001M
1352,1400000US04013010102,"Census Tract 101.02, Maricopa County, Arizona","1,000,000+",***
1355,1400000US04013105002,"Census Tract 1050.02, Maricopa County, Arizona","1,000,000+",***
1356,1400000US04013105003,"Census Tract 1050.03, Maricopa County, Arizona","1,000,000+",***
1357,1400000US04013105004,"Census Tract 1050.04, Maricopa County, Arizona","1,000,000+",***
1358,1400000US04013105103,"Census Tract 1051.03, Maricopa County, Arizona","1,000,000+",***
...,...,...,...,...
69584,1400000US53033005302,"Census Tract 53.02, King County, Washington","1,000,000+",***
69585,1400000US53033024100,"Census Tract 241, King County, Washington","1,000,000+",***
69586,1400000US53033024200,"Census Tract 242, King County, Washington","1,000,000+",***
69587,1400000US53033024602,"Census Tract 246.02, King County, Washington","1,000,000+",***


In [113]:
## replace census tracts with a value label of '1,000,000+' and 10, 000+ with 1,000,000 and 10,000 respectively
value['B25077_001E']=value['B25077_001E'].replace({'1,000,000+': '1000000'})
value['B25077_001E']=value['B25077_001E'].replace({'10,000-': '10000'})

<ipython-input-113-9dd51bb19f11>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  value['B25077_001E']=value['B25077_001E'].replace({'1,000,000+': '1000000'})
<ipython-input-113-9dd51bb19f11>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  value['B25077_001E']=value['B25077_001E'].replace({'10,000-': '10000'})


In [114]:
## confirm that it worked
value.loc[value['B25077_001E']=='1,000,000+']

,GEO_ID,NAME,B25077_001E,B25077_001M


In [115]:
## convert to numeric
value['B25077_001E']=pd.to_numeric(value['B25077_001E'], errors='coerce')

<ipython-input-115-8f7b72838f50>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  value['B25077_001E']=pd.to_numeric(value['B25077_001E'], errors='coerce')


In [116]:
value.dtypes

GEO_ID          object
NAME            object
B25077_001E    float64
B25077_001M     object
dtype: object

In [117]:
## keep only relevant vars
value=value.filter(['GEO_ID','B25077_001E'])

In [118]:
## rename variable
value=value.rename(columns={'B25077_001E':'value'})

In [119]:
## merge onto main dataframe
mREFI=pd.merge(left=mREFI, right=value, how='left', left_on='fips_modified', right_on='GEO_ID', indicator=True)


In [120]:
##check left-only merge
mREFI[mREFI._merge=='left_only']

,state,fips,mrfei,fips_modified,NAME,perc_public_transport,perc_white,perc_black,perc_hispanic,perc_asian,...,perc_snap,inequality,unemploy,below_high_school,college_no_degree,associates_degree,bach_or_more,GEO_ID,value,_merge
7,AL,01001020800,0,1400000US01001020800,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,left_only
22,AL,01003010901,41.1765,1400000US01003010901,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,left_only
23,AL,01003010902,18.1818,1400000US01003010902,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,left_only
25,AL,01003011100,16.6667,1400000US01003011100,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,left_only
31,AL,01003011404,15.7895,1400000US01003011404,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,left_only
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
65312,WY,56029995500,12.5,1400000US56029995500,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,left_only
65321,WY,56035000100,22.2222,1400000US56035000100,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,left_only
65334,WY,56039997600,25,1400000US56039997600,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,left_only
65335,WY,56039997700,10,1400000US56039997700,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,left_only


In [121]:
## remove  merge indicator variable and GEO_ID variable 
mREFI=mREFI.drop(columns=['GEO_ID', '_merge'])

**`No Vehicle`**: here, we will create an attribute that represents the share of the population without a vehicle.

In [122]:
## import data
no_vehicle=pd.read_csv('data/acs_5year_2010/vehicle.csv')
no_vehicle

/Users/mkbelay/opt/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3071: DtypeWarning: Columns (2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


,GEO_ID,NAME,B08201_001E,B08201_001M,B08201_002E,B08201_002M,B08201_003E,B08201_003M,B08201_004E,B08201_004M,...,B08201_026E,B08201_026M,B08201_027E,B08201_027M,B08201_028E,B08201_028M,B08201_029E,B08201_029M,B08201_030E,B08201_030M
0,id,Geographic Area Name,Estimate!!Total,Margin of Error!!Total,Estimate!!Total!!No vehicle available,Margin of Error!!Total!!No vehicle available,Estimate!!Total!!1 vehicle available,Margin of Error!!Total!!1 vehicle available,Estimate!!Total!!2 vehicles available,Margin of Error!!Total!!2 vehicles available,...,Estimate!!Total!!4-or-more-person household!!N...,Margin of Error!!Total!!4-or-more-person house...,Estimate!!Total!!4-or-more-person household!!1...,Margin of Error!!Total!!4-or-more-person house...,Estimate!!Total!!4-or-more-person household!!2...,Margin of Error!!Total!!4-or-more-person house...,Estimate!!Total!!4-or-more-person household!!3...,Margin of Error!!Total!!4-or-more-person house...,Estimate!!Total!!4-or-more-person household!!4...,Margin of Error!!Total!!4-or-more-person house...
1,1400000US01001020100,"Census Tract 201, Autauga County, Alabama",696,72,50,49,100,52,354,86,...,0,119,9,13,75,49,58,39,33,39
2,1400000US01001020200,"Census Tract 202, Autauga County, Alabama",730,85,115,70,274,96,175,60,...,13,20,39,40,66,38,63,37,10,19
3,1400000US01001020300,"Census Tract 203, Autauga County, Alabama",1287,110,101,64,329,90,571,121,...,0,119,0,119,217,59,93,46,42,42
4,1400000US01001020400,"Census Tract 204, Autauga County, Alabama",1839,94,19,12,495,80,802,72,...,0,119,46,25,146,36,118,41,77,24
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
73998,1400000US72127000900,"Census Tract 9, San Juan Municipio, Puerto Rico",1485,142,364,86,756,121,317,93,...,20,21,11,17,29,26,9,14,9,13
73999,1400000US72127001000,"Census Tract 10, San Juan Municipio, Puerto Rico",2414,269,331,114,1501,225,446,130,...,27,43,92,72,83,69,27,44,0,123
74000,1400000US72127001100,"Census Tract 11, San Juan Municipio, Puerto Rico",805,129,184,67,369,108,193,84,...,16,27,10,15,25,29,8,13,0,123
74001,1400000US72127001200,"Census Tract 12, San Juan Municipio, Puerto Rico",1231,146,102,50,511,129,441,111,...,8,13,24,21,159,82,15,16,25,30


In [123]:
## keep relevant variables
no_vehicle=no_vehicle.filter(['B08201_001E', 'B08201_002E', 'GEO_ID'])

In [124]:
## remove first row
no_vehicle=no_vehicle.loc[1:,:]

In [125]:
## check dtypes
no_vehicle.dtypes

B08201_001E    object
B08201_002E    object
GEO_ID         object
dtype: object

In [126]:
## convert to numeric
no_vehicle['B08201_001E']=pd.to_numeric(no_vehicle['B08201_001E'])
no_vehicle['B08201_002E']=pd.to_numeric(no_vehicle['B08201_002E'])

In [127]:
## compute share of population without vehicle
no_vehicle['perc_no_vehicle']=(no_vehicle['B08201_002E']/no_vehicle['B08201_001E'])*100
no_vehicle.head()

,B08201_001E,B08201_002E,GEO_ID,perc_no_vehicle
1,696,50,1400000US01001020100,7.183908
2,730,115,1400000US01001020200,15.753425
3,1287,101,1400000US01001020300,7.847708
4,1839,19,1400000US01001020400,1.033170
5,3741,223,1400000US01001020500,5.960973


In [128]:
## check if denominator is zero and if perc_no_vehicle takes on a NaN value as a result
no_vehicle.loc[no_vehicle['B08201_001E']==0]

,B08201_001E,B08201_002E,GEO_ID,perc_no_vehicle
334,0,0,1400000US01003990000,NaN
399,0,0,1400000US01015981903,NaN
806,0,0,1400000US01097003605,NaN
869,0,0,1400000US01097990000,NaN
1064,0,0,1400000US01117980000,NaN
...,...,...,...,...
73951,0,0,1400000US72113993000,NaN
73956,0,0,1400000US72115990000,NaN
73960,0,0,1400000US72117990400,NaN
73972,0,0,1400000US72119992700,NaN


In [129]:
## remove if denominator is zero (i.e, population is zero)
no_vehicle=no_vehicle.loc[no_vehicle['B08201_001E']!=0]
no_vehicle.head()

,B08201_001E,B08201_002E,GEO_ID,perc_no_vehicle
1,696,50,1400000US01001020100,7.183908
2,730,115,1400000US01001020200,15.753425
3,1287,101,1400000US01001020300,7.847708
4,1839,19,1400000US01001020400,1.033170
5,3741,223,1400000US01001020500,5.960973


In [130]:
# keep relevant variables 
no_vehicle=no_vehicle.filter(['GEO_ID', 'perc_no_vehicle'])

In [131]:
## merge onto main dataframe
mREFI=pd.merge(left=mREFI, right=no_vehicle, how='left', left_on='fips_modified', right_on='GEO_ID', indicator=True)

In [132]:
##check left-only merge
mREFI[mREFI._merge=='left_only']

,state,fips,mrfei,fips_modified,NAME,perc_public_transport,perc_white,perc_black,perc_hispanic,perc_asian,...,inequality,unemploy,below_high_school,college_no_degree,associates_degree,bach_or_more,value,GEO_ID,perc_no_vehicle,_merge
7,AL,01001020800,0,1400000US01001020800,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,left_only
22,AL,01003010901,41.1765,1400000US01003010901,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,left_only
23,AL,01003010902,18.1818,1400000US01003010902,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,left_only
25,AL,01003011100,16.6667,1400000US01003011100,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,left_only
31,AL,01003011404,15.7895,1400000US01003011404,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,left_only
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
65312,WY,56029995500,12.5,1400000US56029995500,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,left_only
65321,WY,56035000100,22.2222,1400000US56035000100,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,left_only
65334,WY,56039997600,25,1400000US56039997600,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,left_only
65335,WY,56039997700,10,1400000US56039997700,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,left_only


In [133]:
## remove  merge indicator variable and GEO_ID variable 
mREFI=mREFI.drop(columns=['GEO_ID', '_merge'])

**`Rural Population`**: here we will calculate the share of the population that lives in the rural part of the census tract

In [134]:
## import data
rural=pd.read_csv('data/decennial_2010/rural_urban_pop.csv')
rural

/Users/mkbelay/opt/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3071: DtypeWarning: Columns (3,4,5,6,7) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


,GEO_ID,NAME,P002001,P002002,P002003,P002004,P002005,P002006
0,id,Geographic Area Name,Total,Total!!Urban,Total!!Urban!!Inside urbanized areas,Total!!Urban!!Inside urban clusters,Total!!Rural,Total!!Not defined for this file
1,1400000US01001020100,"Census Tract 201, Autauga County, Alabama",1912,1594,1594,0,318,0
2,1400000US01001020500,"Census Tract 205, Autauga County, Alabama",10766,10762,10762,0,4,0
3,1400000US01001020300,"Census Tract 203, Autauga County, Alabama",3373,3373,3373,0,0,0
4,1400000US01001020400,"Census Tract 204, Autauga County, Alabama",4386,4386,4386,0,0,0
...,...,...,...,...,...,...,...,...
73998,1400000US72053150301,"Census Tract 1503.01, Fajardo Municipio, Puert...",3798,3798,3798,0,0,0
73999,1400000US72053150302,"Census Tract 1503.02, Fajardo Municipio, Puert...",2185,2185,2185,0,0,0
74000,1400000US72053150602,"Census Tract 1506.02, Fajardo Municipio, Puert...",5925,5280,5280,0,645,0
74001,1400000US72053150400,"Census Tract 1504, Fajardo Municipio, Puerto Rico",4469,4469,4469,0,0,0


In [135]:
## remove first row 
rural=rural.loc[1:,:]

In [136]:
## check dtypes
rural.dtypes

GEO_ID     object
NAME       object
P002001    object
P002002    object
P002003    object
P002004    object
P002005    object
P002006    object
dtype: object

In [137]:
## convert variable dtypes
rural['P002002']=pd.to_numeric(rural['P002002'])## urban 
rural['P002005']=pd.to_numeric(rural['P002005'])## rural

<ipython-input-137-7a33e916ebf9>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  rural['P002002']=pd.to_numeric(rural['P002002'])## urban
<ipython-input-137-7a33e916ebf9>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  rural['P002005']=pd.to_numeric(rural['P002005'])## rural


In [138]:
## sum rural and urban population to get total population
rural['total_pop']=rural['P002002']+rural['P002005']

<ipython-input-138-2aa23e7fea92>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  rural['total_pop']=rural['P002002']+rural['P002005']


In [139]:
## keep relevant variables
rural=rural.filter(['P002005', 'total_pop', 'GEO_ID'])

In [140]:
## calculate share of population that is rural
rural['perc_rural']=(rural['P002005']/rural['total_pop'])*100
rural

,P002005,total_pop,GEO_ID,perc_rural
1,318,1912,1400000US01001020100,16.631799
2,4,10766,1400000US01001020500,0.037154
3,0,3373,1400000US01001020300,0.000000
4,0,4386,1400000US01001020400,0.000000
5,0,2170,1400000US01001020200,0.000000
...,...,...,...,...
73998,0,3798,1400000US72053150301,0.000000
73999,0,2185,1400000US72053150302,0.000000
74000,645,5925,1400000US72053150602,10.886076
74001,0,4469,1400000US72053150400,0.000000


In [141]:
## check for instances where denominator is zero
rural.loc[rural['total_pop']==0]

,P002005,total_pop,GEO_ID,perc_rural
69,0,0,1400000US01097990000,NaN
95,0,0,1400000US01003990000,NaN
272,0,0,1400000US01015981903,NaN
1617,0,0,1400000US04012980000,NaN
2592,0,0,1400000US04013980100,NaN
...,...,...,...,...
73937,0,0,1400000US72089990001,NaN
73948,0,0,1400000US72091990025,NaN
73953,0,0,1400000US72095990000,NaN
73980,0,0,1400000US72097990000,NaN


In [142]:
# remove observations with zero population
rural=rural.loc[rural['total_pop']!=0]
rural.head()

,P002005,total_pop,GEO_ID,perc_rural
1,318,1912,1400000US01001020100,16.631799
2,4,10766,1400000US01001020500,0.037154
3,0,3373,1400000US01001020300,0.000000
4,0,4386,1400000US01001020400,0.000000
5,0,2170,1400000US01001020200,0.000000


In [143]:
# keep relevant variables 
rural=rural.filter(['GEO_ID', 'perc_rural'])

In [144]:
## merge onto main dataframe
mREFI=pd.merge(left=mREFI, right=rural, how='left', left_on='fips_modified', right_on='GEO_ID', indicator=True)

In [145]:
##check left-only merge
mREFI[mREFI._merge=='left_only']

,state,fips,mrfei,fips_modified,NAME,perc_public_transport,perc_white,perc_black,perc_hispanic,perc_asian,...,unemploy,below_high_school,college_no_degree,associates_degree,bach_or_more,value,perc_no_vehicle,GEO_ID,perc_rural,_merge
7,AL,01001020800,0,1400000US01001020800,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,left_only
22,AL,01003010901,41.1765,1400000US01003010901,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,left_only
23,AL,01003010902,18.1818,1400000US01003010902,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,left_only
25,AL,01003011100,16.6667,1400000US01003011100,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,left_only
31,AL,01003011404,15.7895,1400000US01003011404,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,left_only
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
65312,WY,56029995500,12.5,1400000US56029995500,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,left_only
65321,WY,56035000100,22.2222,1400000US56035000100,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,left_only
65334,WY,56039997600,25,1400000US56039997600,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,left_only
65335,WY,56039997700,10,1400000US56039997700,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,left_only


In [146]:
## remove  merge indicator variable and GEO_ID variable 
mREFI=mREFI.drop(columns=['GEO_ID', '_merge'])

`Export dataframe to csv`

Export the dataframe containing all the variables to csv file which will be used for data characterizing, pre-processing, and modeling. 

In [147]:
## dataframe dimensions
mREFI.shape

(65345, 26)

In [148]:
## check columns
mREFI.columns

Index(['state', 'fips', 'mrfei', 'fips_modified', 'NAME',
       'perc_public_transport', 'perc_white', 'perc_black', 'perc_hispanic',
       'perc_asian', 'perc_native', 'perc_pacific', 'median_income', 'SQMI',
       'pop_density', 'perc_pov', 'perc_snap', 'inequality', 'unemploy',
       'below_high_school', 'college_no_degree', 'associates_degree',
       'bach_or_more', 'value', 'perc_no_vehicle', 'perc_rural'],
      dtype='object')

In [149]:
## remove FIPS code column
mREFI=mREFI.drop(columns=['fips'])

In [150]:
## check dataframe
mREFI.head()

,state,mrfei,fips_modified,NAME,perc_public_transport,perc_white,perc_black,perc_hispanic,perc_asian,perc_native,...,perc_snap,inequality,unemploy,below_high_school,college_no_degree,associates_degree,bach_or_more,value,perc_no_vehicle,perc_rural
0,AL,16.6667,1400000US01001020100,"Census Tract 201, Autauga County, Alabama",4.651163,78.717523,16.196794,0.829187,0.000000,0.000000,...,5.747126,0.513,3.8,10.5,17.7,6.0,25.4,120700.0,7.183908,16.631799
1,AL,16.6667,1400000US01001020200,"Census Tract 202, Autauga County, Alabama",0.000000,38.465347,59.158416,0.297030,2.079208,0.000000,...,8.904110,0.386,7.9,14.9,17.2,3.3,26.7,138500.0,15.753425,0.000000
2,AL,17.6471,1400000US01001020300,"Census Tract 203, Autauga County, Alabama",0.907912,82.020886,16.596105,1.100762,0.282247,0.000000,...,9.712510,0.405,5.7,9.5,25.0,8.5,16.4,111300.0,7.847708,0.000000
3,AL,13.3333,1400000US01001020400,"Census Tract 204, Autauga County, Alabama",0.000000,93.863636,2.314050,2.644628,0.392562,0.000000,...,6.688418,0.357,4.5,6.2,24.5,6.5,25.3,126300.0,1.033170,0.000000
4,AL,6.25,1400000US01001020500,"Census Tract 205, Autauga County, Alabama",0.000000,80.177098,11.581807,4.739384,1.730731,0.201248,...,3.742315,0.334,3.7,2.8,22.4,6.7,37.5,173000.0,5.960973,0.037154


In [151]:
## export to csv
mREFI.to_csv('combined_data.csv', index=False)